In [1]:
import cv2 as cv
from PIL import ImageGrab
from dataclasses import dataclass
import numpy as np
from matplotlib import pyplot as plt
import math
import pyautogui
from pynput import keyboard, mouse
import time

In [2]:
@dataclass
class Coordinate:
    x: int
    y: int

    def __add__(self, other):
        return Coordinate(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Coordinate(self.x - other.x, self.y - other.y)

    def asTuple(self):
        return (self.x, self.y)

In [3]:
def anotate():
    mouse_listener = None
    keyboard_listener = None
    
    active_dist = None
    dists_by_time = []
    
    click_start = None
    def on_click(x, y, button, pressed):
        nonlocal click_start
        nonlocal active_dist
        nonlocal dists_by_time
    
        if active_dist is None:
            return
        
        if button == mouse.Button.left:
            if pressed:
                click_start = time.time()
            else:
                click_end = time.time()
                duration = click_end - click_start
    
                dists_by_time.append((duration, active_dist))
                active_dist = None
    
    start = None
    end = None
    def on_press(key):
        nonlocal mouse_listener
        nonlocal keyboard_listener
        
        nonlocal start
        nonlocal end
    
        nonlocal active_dist
    
        def kill():
            mouse_listener.stop()
            keyboard_listener.stop()
    
        try:
            if key.char == "f":
                x, y = pyautogui.position()
    
                if start is None:
                    start = Coordinate(x,y)
                elif end is None:
                    end = Coordinate(x,y)
                    if active_dist is not None:
                        kill()
                        
                    active_dist = math.sqrt((end.x - start.x)**2 + (end.y - start.y)**2)
                else:
                    start_coord = Coordinate(x,y)
                    end_coord = None
    
                if start is not None and end is not None:
                    print(f"Ready, start={start.asTuple()}, end={end.asTuple()}")
                    
        except AttributeError:
            if key == keyboard.Key.esc:
                kill()
    
    mouse_listener = mouse.Listener(on_click=on_click)
    keyboard_listener = keyboard.Listener(on_press=on_press)
    
    mouse_listener.start()
    keyboard_listener.start()
    
    mouse_listener.join()
    keyboard_listener.join()

    print(dists_by_time)
    return dists_by_time

In [27]:
anotate()

Ready, start=(480, 442), end=(831, 446)
{23.50317692756653: 351.02279128284533}


{23.50317692756653: 351.02279128284533}

In [28]:
anotate()

Ready, start=(689, 452), end=(339, 449)
{23.510619163513184: 350.0128569067142}


{23.510619163513184: 350.0128569067142}

In [5]:
data = [
    (23.50317692756653, 351.02279128284533),
    (23.510619163513184, 350.0128569067142)]

In [8]:
for duration, dist in data:
    print(f"pps: {dist/duration}")

pps: 14.935120999371616
pps: 14.887436799193676
